### checking the number of positron events totalling in each port via the MC

idk why i didnt check this earlier, should have been obvious

In [1]:
# import stuff
import sys,os,os.path

sys.path.append("../../")   # cite IC from parent directory
sys.path.append("/home/e78368jw/Documents/NEXT_CODE/next_misc/")
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'
import IC.invisible_cities.io.dst_io                           as     dstio
import IC.invisible_cities.io.mcinfo_io as mcio
from    IC.invisible_cities.core.core_functions   import shift_to_bin_centers

import core.functions as func
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy.special as special
from scipy.stats import skewnorm
from scipy.optimize import curve_fit

from scipy.integrate import quad

In [2]:
def collate_ports(path_array):
    '''
    Collect individual ports and merge the information

    Args:
        path_array          :           an array of folder paths to h5 files
                                        respective of the multiple ports

    
    Returns:
        array               :           output of collective ports
    '''

    # strip array except from port information
    port_id = [x.split('PORT_')[1][:2] for x in path_array]

    for i in range(len(path_array)):
        print("Porting {}".format(path_array[i]))
        if (i==0):
            x = (func.load_data(path_array[i]))
            tracks = x[0]
            #particles = x[1]
            # match tracks for particle
            #particles['event_id'] = particles['event_id'] * ((i+1)*2)
            # add on the column for port ID
            tracks['PORT'] = str(port_id[i])
        else:
            nu_x = (func.load_data(path_array[i]))
            nu_tracks = nu_x[0]
            #nu_particles = nu_x[1]
            print("Tracks: {}".format(func.len_events(nu_tracks)))
            # multiply the events numbers to avoid overlap
            nu_tracks['event'] = nu_tracks['event'] * (i+1)
            nu_tracks['PORT'] = str(port_id[i])
            #nu_particles['event_id'] = nu_particles['event_id'] * ((i+1)*2)


            tracks = tracks.append(nu_tracks)
            #particles = particles.append(nu_particles)

        print("Done! Tracks available: {}".format(func.len_events(tracks)))
    
    return tracks

def positron_ports(path_array):
    '''
    Collect positron events from multiple ports.
    Bespoke, use with care.
    '''

    for i in range(len(path_array)):
        print("Loading positrons from {}".format(path_array[i]))
        if (i==0):
            posi = func.positron_scraper(path_array[i])
            # multiply the event numbers to match track values
            posi['event_id'] = posi['event_id'] * ((i+1)*2)
        else:
            nu_posi = func.positron_scraper(path_array[i])
            print("Positron events: {}".format(func.len_events(nu_posi, tag = 'event_id')))
            # multiple the event numbers to avoid overlap, the *2 is to match it with the tracking values
            nu_posi['event_id'] = nu_posi['event_id'] * ((i+1)*2)
            posi = posi.append(nu_posi)
        print("Port finished! Tracks available: {}".format(func.len_events(posi, tag = 'event_id')))
    
    return posi


# a modified version of the positron scraper
def MC_collector(data_path, event_list, save = False):
    '''
    a modified version of the positron scraper
    that only collects events found within the event list
    this only works for port 1a at the moment, because I
    would need to add an event/port mapping function and im too lazy
    '''



     # collect all filenames
    try:
        file_names = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and f.endswith('.h5')]
    except:
        print("File path incorrect, please state the correct file path\n(but not any particular folder!)")


    # read in a singular file to collect the column titles
    
    MC_df_single = pd.read_hdf(data_path + file_names[0], 'MC/particles')

    MC_df = []
    pos_df = pd.DataFrame(columns = MC_df_single.columns)
    eventmap = []


    i = 0
    

    # how much you chunk your data
    chunker = np.floor(len(file_names)*0.1)

    # chunk file_names
    
    for file in file_names:
        file_path = data_path + file

        # load in file
        MC_df_temp = pd.read_hdf(file_path, 'MC/particles')
        MC_df.append(MC_df_temp)
        eventmap.append(mcio.load_eventnumbermap(file_path).set_index('nexus_evt'))


        i += 1

        # chunk checker, every time you hit a certain chunk,
        # collect the positron events and wipe the df
        if ((i%chunker) == 0):
            #print("Chunking at event {}!".format(i))
            # concat the list
            MC_df = pd.concat(MC_df, axis = 0, ignore_index = True)
            #print("Post concat")
            #display(MC_df)
            # alter all the event list numbers by multiples of 2
            MC_df['event_id'] = MC_df['event_id'] * 2
            pos_data = MC_df[MC_df['event_id'].isin(event_list)]

            
            #display(pos_data)
            #print(type(pos_data))
            # collect positron events into df
            pos_df = pos_df.append(pos_data)
            #print("{} positron events found\n{} positron events total".format(pos_data.shape[0],pos_df.shape[0]))
            #display(pos_df)

            # make space
            MC_df = []

    if (save == True):
        pos_df.to_hdf('positrons.h5', key = 'pos', mode = 'w')

    return pos_df


In [4]:
folder_paths = ['../FOM_merge&fit/75_9e-3/PORT_1a/isaura/', '../FOM_merge&fit/75_9e-3/PORT_1b/isaura/',  '../FOM_merge&fit/75_9e-3/PORT_2a/isaura/',  '../FOM_merge&fit/75_9e-3/PORT_2b/isaura/']


In [ ]:
# THIS DOESNT WORK HOW YOU WANT, BUT ITS CLOSE!
# YOU NEED SOMETHING THAT COLLECTS ONLY POSITRON EVENTS THAT OUTPUT
# GAMMAS THAT ESCAPE I THINK
for i in range(len(folder_paths)):
    particles = MC_collector(folder_paths[i], merge_events_np)